#Setup Kapture-localisation environment

In [ ]:
!sudo apt-get install \
    git \
    cmake \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libfreeimage-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev
!sudo apt-get install libatlas-base-dev libsuitesparse-dev
%cd /content/
!git clone https://ceres-solver.googlesource.com/ceres-solver
%cd /content/ceres-solver/
!git checkout $(git describe --tags) # Checkout the latest release
!mkdir build
%cd /content/ceres-solver/build
!cmake .. -DBUILD_TESTING=OFF -DBUILD_EXAMPLES=OFF
!make -j
!sudo make install
%cd /content/
!git clone https://github.com/colmap/colmap.git
%cd /content/colmap/
!git checkout dev
!mkdir build
%cd /content/colmap/build
!cmake ..
!make -j
!sudo make install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libboost-filesystem-dev is already the newest version (1.65.1.0ubuntu1).
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libboost-system-dev is already the newest version (1.65.1.0ubuntu1).
libsuitesparse-dev is already the newest version (1:5.1.2-2).
libboost-graph-dev is already the newest version (1.65.1.0ubuntu1).
libboost-test-dev is already the newest version (1.65.1.0ubuntu1).
libcgal-dev is already the newest version (4.11-2build1).
libeigen3-dev is already the newest version (3.3.4-4).
libgflags-dev is already the newest version (2.2.1-1).
libglew-dev is already the newest version (2.0.0-5).
libgoogle-glog-dev is already the newest version (0.3.5-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
git is already the newest version (1:2.17.1-1ubuntu0.8).
libqt5opengl5-dev is alre

In [ ]:
!pip install -U numpy
!sudo apt-get install -y pandoc asciidoctor
!sudo apt-get install -y git python3.6 python3-pip colmap

In [ ]:
%cd /content/
!git clone https://github.com/naver/kapture.git
%cd /content/kapture/
!ls
!python3 -m unittest discover -s tests
# install
!python3 setup.py build install

In [ ]:
# copy your image folder to kapture dir
!cp -R /content/drive/MyDrive/3D_imperial_test_results/Output/Test_EEE/kapture_input /content/kapture/tools/

In [ ]:
# convert your image folder to kapture format
%cd /content/kapture/tools

! python kapture_import_image_folder.py -i ./kapture_input \
  -o ./kapture_output

/content/kapture/tools


In [ ]:
# change kapture data format
%cd /content/kapture/tools/kapture_output/sensors/
import os
import glob

my_path='/content/kapture/tools/kapture_output/sensors/records_data/'
files=sorted(glob.glob(my_path+'*.png'))
image_name = []
for obj in files:
  obj_content = obj.split('/')
  obj_last = obj_content[-1]
  image_name.append(obj_last)
print(image_name)
fout = open("records_camera.txt", "a")
fout.write('# kapture format: 1.1\n')
fout.write('# timestamp, device_id, image_path\n')
for image in image_name:
  str_content = image.split('_')
  # print(str_content)
  timestamp = str_content[1]
  cam_id = str_content[2][0]
  if cam_id == '0':
  # print(timestamp)
    output0 = str(timestamp+', '+cam_id+', '+image+'\n')
    # image_content = image.split('.')
    # modefied_img = image_content[0][:-1]+'1.'+image_content[1]
    # output1 = str(timestamp+', 1'+', '+modefied_img+'\n')
    fout.write(output0)
    # fout.write(output1)
  elif cam_id == '1':
    output1 = str(timestamp+', '+cam_id+', '+image+'\n')
    fout.write(output1)
    # print(output0)
    # print(output1)
  # output0 = str(timestamp+', '+cam_id+', '+image+'\n')
  # fout.write(output0)
fout.close()

#open and read the file after the appending:
f = open("/content/kapture/tools/kapture_output/sensors/records_camera.txt", "r")
# # print(f.read())
lines = f.readlines()
count = len(lines)
f.close()
for i in range (count):
  # read sensor
  fin = open("/content/kapture/tools/kapture_output/sensors/records_camera.txt", "r")
  fout = open("/content/kapture/tools/kapture_output/sensors/records_camera1.txt", "a")

  lines = fin.readlines()
  # print(lines[i])
  if i<2:
    line = lines[i]
    fout.write(line)
  else:
    line = lines[i]
    print(line)
    target='' 
    values = line.split(', ')
    for i in range (len(values)):
      if i == 1:
        target=values[i]

    line = line.replace(target,'sensor0')
    # print(line)
    fout.write(line)
  #rewrite sensor  
  
  # lines = f.readlines()
  fin.close()
  fout.close()
  # delete records_camera.txt and change records_camera1.txt to records_camera.txt

In [ ]:
# extract r2d2 features
%cd /content/
!git clone https://github.com/naver/r2d2.git
%cd /content/r2d2/
!python extract_kapture.py --model ./models/r2d2_WASF_N8_big.pt --kapture-root /content/drive/MyDrive/3D_imperial_test_results/Output/Test_EEE/kapture_output/mapping --top-k 5000 --scale-f=1.1892 --min-scale=0.3 --max-scale=1 --min-size=128 --max-size=9999 --reliability-thr=0.7 --repeatability-thr=0.7
# !python extract_kapture.py -h

/content
Cloning into 'r2d2'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 97 (delta 0), reused 1 (delta 0), pack-reused 94
Unpacking objects: 100% (97/97), done.


In [ ]:
# deep image retrieval
%cd /content/
!git clone https://github.com/naver/deep-image-retrieval.git
%cd /content/deep-image-retrieval/
!python -m dirtorch.extract_kapture --kapture-root /content/drive/MyDrive/3D_imperial_test_results/Output/Test_EEE/kapture_output/mapping --checkpoint /content/drive/MyDrive/3D_imperial_test_results/global_feature_net/Resnet101-AP-GeM-LM18.pt --gpu 0


/content
Cloning into 'deep-image-retrieval'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 205 (delta 12), reused 9 (delta 3), pack-reused 179
Receiving objects: 100% (205/205), 108.09 KiB | 9.83 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [ ]:
# rearrange your structure
%cd /content/drive/MyDrive/3D_imperial_test_results/Output/Test_EEE/kapture_output/mapping
!mv ./reconstruction ./local_features
!mv ./local_features/global_features ..
!mkdir ./local_features/r2d2_WASF_N8_big
!mv  ./local_features/descriptors ./local_features/r2d2_WASF_N8_big
!mv  ./local_features/keypoints ./local_features/r2d2_WASF_N8_big
!mv ./local_features/ ..

/content/drive/MyDrive/3D_imperial_test_results/Output/Test_EEE/kapture_output/mapping


In [ ]:
%cd /content/
!git clone https://github.com/naver/kapture-localization.git

/content
Cloning into 'kapture-localization'...
remote: Enumerating objects: 832, done.
remote: Counting objects: 100% (368/368), done.
remote: Compressing objects: 100% (275/275), done.
remote: Total 832 (delta 207), reused 162 (delta 72), pack-reused 464
Receiving objects: 100% (832/832), 7.50 MiB | 34.14 MiB/s, done.
Resolving deltas: 100% (397/397), done.


In [ ]:
%cd /content/kapture-localization
# check everything is fine
!python3 -m unittest discover -s tests
# install
!python3 setup.py build install

/content/kapture-localization
..
----------------------------------------------------------------------
Ran 2 tests in 0.745s

OK
running build
running build_py
creating build
creating build/lib
creating build/lib/kapture_localization
copying kapture_localization/__init__.py -> build/lib/kapture_localization
copying kapture_localization/evaluation.py -> build/lib/kapture_localization
creating build/lib/kapture_localization/matching
copying kapture_localization/matching/matching.py -> build/lib/kapture_localization/matching
copying kapture_localization/matching/__init__.py -> build/lib/kapture_localization/matching
creating build/lib/kapture_localization/image_retrieval
copying kapture_localization/image_retrieval/pairing.py -> build/lib/kapture_localization/image_retrieval
copying kapture_localization/image_retrieval/__init__.py -> build/lib/kapture_localization/image_retrieval
creating build/lib/kapture_localization/colmap
copying kapture_localization/colmap/__init__.py -> build/lib/k

In [ ]:
%cd /content/drive/MyDrive/3D_imperial_test_results/Output/Test_EEE/kapture_output/
! kapture_pipeline_mapping.py -i ./mapping/ \-kpt ./local_features/r2d2_WASF_N8_big/keypoints/r2d2_WASF_N8_big \-desc ./local_features/r2d2_WASF_N8_big/descriptors/r2d2_WASF_N8_big \-gfeat ./global_features/Resnet101-AP-GeM-LM18 \-matches ./local_features/r2d2_WASF_N8_big/NN_no_gv/matches \-matches-gv ./local_features/r2d2_WASF_N8_big/NN_colmap_gv/matches \--colmap-map ./colmap-sfm/r2d2_WASF_N8_big/Resnet101-AP-GeM-LM18_top20 \--topk 20 

Streaming output truncated to the last 5000 lines.
   8  7.668456e+04    9.95e-01    1.96e+06   2.64e+00   9.41e-01  2.69e+07        1    7.66e-01    4.86e+00
   9  7.668432e+04    2.36e-01    9.06e+06   1.83e+00   9.57e-01  8.06e+07        1    5.46e-01    5.41e+00
  10  7.668429e+04    3.41e-02    5.40e+06   5.06e-01   1.02e+00  2.42e+08        1    5.61e-01    5.97e+00
  11  7.668429e+04    0.00e+00    5.40e+06   0.00e+00   0.00e+00  1.21e+08        1    1.22e-01    6.09e+00
  12  7.668429e+04    0.00e+00    5.40e+06   0.00e+00   0.00e+00  3.02e+07        1    1.14e-01    6.21e+00
  13  7.668428e+04    2.63e-03    2.11e+06   5.80e-02   1.01e+00  9.06e+07        1    6.45e-01    6.85e+00
  14  7.668428e+04    0.00e+00    2.11e+06   0.00e+00   0.00e+00  4.53e+07        1    1.34e-01    6.99e+00
  15  7.668428e+04    3.27e-03    3.12e+06   1.72e-02   1.00e+00  1.36e+08        1    5.58e-01    7.55e+00
  16  7.668428e+04    0.00e+00    3.12e+06   0.00e+00   0.00e+00  6.80e+07        1  